This notebook contains code made for deriving snow depths from the ICESat-2 ATL03 Dataset. 

Workflow:

- Estimate surface height from ICESat-2 ATL03 data

Required input data: 

- ATL03 Trackline
- Digital Elevation Model
- Masks (Optional) 



In [ ]:
## Package Import ##

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import h5py  
import os
import fiona
import geopandas
import time
from sklearn.neighbors import KernelDensity
import scipy
from astropy.time import Time
print(geopandas.__version__) # Version 0.8.1

import rasterio
import geopandas as gpd
import pyproj
import rasterio
import glob
from numpy import *

getATL03 function loads in raw ICESat-2 ATL03 data in .h5 format and outputs are geodataframe where each row represents a photon.

In [ ]:
def getATL03(f,beam):
    # height of each received photon, relative to the WGS-84 ellipsoid (with some, not all corrections applied, see background info above)
    heights=f[beam]['heights']['h_ph'][:]
    # latitude (decimal degrees) of each received photon
    lats=f[beam]['heights']['lat_ph'][:]
    # longitude (decimal degrees) of each received photon
    lons=f[beam]['heights']['lon_ph'][:]
    # seconds from ATLAS Standard Data Product Epoch. use the epoch parameter to convert to gps time
    dt=f[beam]['heights']['delta_time'][:]
    # confidence level associated with each photon event
    # -2: TEP
    # -1: Events not associated with a specific surface type
    #  0: noise
    #  1: buffer but algorithm classifies as background
    #  2: low
    #  3: medium
    #  4: high
    # Surface types for signal classification confidence
    # 0=Land; 1=Ocean; 2=SeaIce; 3=LandIce; 4=InlandWater    
    conf=f[beam]['heights']['signal_conf_ph'][:,0] #choose column 2 for confidence of sea ice photons
    # number of ATL03 20m segments
    n_seg, = f[beam]['geolocation']['segment_id'].shape
    #GEOID
    geoid = f[beam]['geophys_corr']['geoid'][:]
    # first photon in the segment (convert to 0-based indexing)
    Segment_Index_begin = f[beam]['geolocation']['ph_index_beg'][:] - 1
    # number of photon events in the segment
    Segment_PE_count = f[beam]['geolocation']['segment_ph_cnt'][:]
    # along-track distance for each ATL03 segment
    Segment_Distance = f[beam]['geolocation']['segment_dist_x'][:]
    # along-track distance (x) for photon events
    x_atc = np.copy(f[beam]['heights']['dist_ph_along'][:])
    # cross-track distance (y) for photon events
    y_atc = np.copy(f[beam]['heights']['dist_ph_across'][:])

    for j in range(n_seg):
        # index for 20m segment j
        idx = Segment_Index_begin[j]
        # number of photons in 20m segment
        cnt = Segment_PE_count[j]
        # add segment distance to along-track coordinates
        x_atc[idx:idx+cnt] += Segment_Distance[j]
        #geoid
        #geoid[idx:idx+cnt] += geoid[j]


    df03=pd.DataFrame({'lats':lats,'lons':lons,'x':x_atc,'y':y_atc,'heights':heights,'dt':dt,'conf':conf})
    return df03

In [ ]:
def load_ATL03(path, date, beam):
    ATL03_path = path + date + '/ATL03/'
    for file in os.listdir(ATL03_path):
        if file.endswith('.h5'):
            fname = file
    f = h5py.File(ATL03_path+fname,'r')
    geoid = f[beam]['geophys_corr']['geoid'][:]
    #10m res
    geoid = list(np.repeat(geoid, 2))
    #100m res
    #geoid = geoid[::5]
    #geoid = geoid.tolist()
    df = getATL03(f,beam)
    df = df[df['conf'] > 2]
    df['AT_dist']=df.x-df.x.values[0]
    return df, geoid

Surf function estimates the surface height by grouping photons height values along-track, using kernel density to estimate surface height, and substracting the geoid. 

In [ ]:
def surface(dfATL03, window_width, geoid, resolution):
    
    '''
    Input:
        dfATL03:
        window_width:
        geoid:
        resolution:
    Output:
        df:
    '''
    
    # Track function time
    startTime = time.time()
    
    # Along track distance values, min and max to determine boundries for moving window.
    AT_dist_values = dfATL03['AT_dist'].values
    AT_dist_totmax = AT_dist_values.max()
    AT_dist_totmin = AT_dist_values.min()
    
    
    # Photon height level array
    heights_values = dfATL03['heights'].values
    
    # Confidence level array
    conf_values = dfATL03['conf'].values
    
    # Coordinates array
    lats = dfATL03['lats'].values
    lons = dfATL03['lons'].values
    
    # Moving Window - List of values starting from Along track distance min to max,
    # at a resulution of either 10 or 100 meters
    windows = np.arange(AT_dist_totmin, AT_dist_totmax, resolution).tolist()


    # Empty arrays to store new along track segmented values instead of per-photon.
    
    #Along track window center array
    AT_window_center_arr = np.empty(len(windows))
    
    # Surface array
    surface_arr = np.empty(len(windows))  
    
    # Latitude
    lat_arr = np.empty(len(windows))  
    
    # Longitude
    lon_arr = np.empty(len(windows))  
    
    # track number of iterations
    i = 0
    
    # Iterate through every photon(row), store then into segments at x resolution and calculate surface
    # height based on photon height density.
    for window_center in windows:    
 
        # Get minimum window boundries
        min_dist = window_center - window_width
        min_dist_array = np.where(AT_dist_values > min_dist)
        min_dist_row = min_dist_array[0][0]

        # Get maximum window boundries
        max_dist = window_center + window_width
        if max_dist < AT_dist_values[-1]:
            max_dist_array = np.where(AT_dist_values > max_dist)
            max_dist_row = max_dist_array[0][0]
        else:
            max_dist = AT_dist_values[-1] 

        # Get window center lat & long (for plotting)     
        idx = (np.abs(AT_dist_values - window_center)).argmin()
        lat = lats[idx]
        lon = lons[idx]

        # Select photons AT_dist & heights within window boundries
        window_heights = heights_values[min_dist_row:max_dist_row]

        # Only Search for surface if there is enough Photons in window
        if len(window_heights) > 10: #Normally 5
            
            binwidth = 0.2
            
            # Kernel Density Function
            kde = KernelDensity(kernel='gaussian', bandwidth=binwidth).fit(window_heights[:, None])
            
            # Kernel density scores
            kde_scores = kde.score_samples(window_heights[:, None])
            
            # Select bin with heighest density
            maxdensityI = np.max(kde_scores)
            
            # Extract height from heighest density bin
            max_index = np.where(kde_scores == maxdensityI)
            maxdensityH =  window_heights[max_index]
            
            # Define surface-height as height with highest photon density  
            Surface_Estimation = maxdensityH[0]
            
            # Add center of window as new along track distance in element i
            AT_window_center_arr[i] = window_center 
            
            # Add estimated surface
            surface_arr[i] = Surface_Estimation
            
            # Add Latitude
            lat_arr[i] = lat
            
            # Add longitude
            lon_arr[i] = lon
            
            i+=1

        else:
            
            # If there is not enough photons in window, set surface to 0. 
            AT_window_center_arr[i] = window_center
            surface_arr[i] = 0
            lat_arr[i] = lat
            lon_arr[i] = lon     
            i+=1
        
        # Calculate and print progress
        wincen = int(window_center)
        if wincen % 25000 < 1:  
            m = int(i / len(windows) * 100)
            print(m, "% Done")
    
    # Create new segmented dataframe at x resolution with surface heights, along track distance and coordinates. 
    df_surface = pd.DataFrame(AT_window_center_arr, columns = ['AT_dist']) 
    df_surface["Surface"] = surface_arr
    df_surface["lats"] = lat_arr
    df_surface["lons"] = lon_arr

    # Add geoid to correct surface heights when comparing heights to Digital Elevation Models with a different coordinate system.
    if len(geoid) < len(df_surface.index):
      while len(geoid) < len(df_surface.index):
        geoid.append(geoid[-1])
      df_surface["geoid"] = geoid
    elif len(geoid) > len(df_surface.index):
       df_surface["geoid"] = geoid[:len(df_surface.index)]   
    else:
      df_surface["geoid"] = geoid

    # Calculate corrected surface height
    df_surface['Surface_corr'] = (df_surface['Surface'] - df_surface['geoid']).round(2)
    
    # Script Runtime
    runTime =  int(time.time() - startTime)
    runTimeMin = runTime/60
    runTimeSec = runTime%60
    print("\nScript Runtime: %i minutes and %i seconds" % (runTimeMin,runTimeSec))
    
    return df_surface

df_to_shp converts every estimated surface height to a point in a shapefile. These points are used further on for snow depth estimations

In [ ]:
def snow_depth(df_surface, beam, DEM):
        
    '''
    Input:
        df_surface:
        beam:
        DEM:
    Output:
        pts:
    '''
    #Create Geodataframe from dataframe
    df_sd = df_surface
    df_sd = geopandas.GeoDataFrame(
    df_sd, geometry=geopandas.points_from_xy(df_sd.lons, df_sd.lats))

    #Set coordinate system
    df_sd = df_sd.set_crs('EPSG:4326')

    #Convert coordinate system
    df_sd = df_sd.to_crs(25833)       

    #Sample DEM to Icesat-2 points
    coords = [(x,y) for x, y in zip(df_sd.geometry.x, df_sd.geometry.y)]
    src = rasterio.open(DEM)
    df_sd['DEM'] = [x[0] for x in src.sample(coords)]

    #Norwegian DEM
    df_sd['SD'] = round(df_sd.Surface_corr - df_sd.DEM,2)

    #Arctic DEM - Using same coordinate system as IS2, so no need to correct.
    #pts['SD'] = round(pts.Surface - pts.DEM,2)

    print('Training Points: ',df_sd.shape[0])

    return df_sd

In [ ]:
def Mask(df_sd, path, date):
    
    df_sd_masked = df_sd
    coords = [(x,y) for x, y in zip(df_sd_masked.geometry.x, df_sd_masked.geometry.y)]

    SLOPE = path[:-7] + '/Masks/Slope.tif'
    Binary_Mask = path + date + '/Masks/Binary_Mask.tif'
    listofmasks=[SLOPE, Binary_Mask]
    listofmasksname=['SLOPE', 'Binary_Mask']
    
    i = 0
    for mask in listofmasks:
        src = rasterio.open(mask)
        df_sd_masked[listofmasksname[i]] = [x[0] for x in src.sample(coords)]
        i=i+1
        
    
    print('Training Points: ',df_sd_masked.shape[0])

    #Mask out points on slope over 8 degrees
    df_sd_masked = df_sd_masked[df_sd_masked['SLOPE'] < 8]
    print('Slope: ',df_sd_masked.shape[0])
    
    df_sd_masked = df_sd_masked[df_sd_masked['Binary_Mask'] > 0]
    print('Binary Mask: ',df_sd_masked.shape[0])

    
    print('Outliers: ',df_sd_masked.shape[0])

    print('Training data points left after masking: ', df_sd_masked.shape[0])
    
    return df_sd_masked
        

In [ ]:
def Rescale(df_sd_masked, window_width, rescale_resolution, beam, path, date):

    # Along track distance values, min and max to determine boundries for moving window.
    AT_dist_values = df_sd_masked['AT_dist'].values
    AT_dist_totmax = AT_dist_values.max()
    AT_dist_totmin = AT_dist_values.min()

    # Coordinates
    df_sd_masked['lons'] = df_sd_masked.geometry.x
    df_sd_masked['lats']= df_sd_masked.geometry.y

    # Coordinates array
    lats = df_sd_masked['lats'].values
    lons = df_sd_masked['lons'].values

    # Snow depth array
    SD_values = df_sd_masked['SD'].values


    # Moving Window - List of values starting from Along track distance min to max,
    # at a resulution of x meters.
    windows = np.arange(AT_dist_totmin, AT_dist_totmax, rescale_resolution).tolist()

    #Along track window center array
    AT_window_center_arr = np.empty(len(windows))
    
    # Snow depth array
    snow_depth_arr = np.empty(len(windows))  
    
    # Latitude
    lat_arr = np.empty(len(windows))  
    
    # Longitude
    lon_arr = np.empty(len(windows)) 

    # track number of iterations
    i = 0
    
    # Iterate through every photon(row), store then into segments at x resolution and calculate surface
    # height based on photon height density.
    for window_center in windows:    

        # Get minimum window boundries
        min_dist = window_center - window_width
        min_dist_array = np.where(AT_dist_values > min_dist)
        min_dist_row = min_dist_array[0][0]

        # Get maximum window boundries
        max_dist = window_center + window_width
        if max_dist < AT_dist_values[-1]:
            max_dist_array = np.where(AT_dist_values > max_dist)
            max_dist_row = max_dist_array[0][0]
        else:
            max_dist = AT_dist_values[-1] 

        # Get window center lat & long (for plotting)     
        idx = (np.abs(AT_dist_values - window_center)).argmin()
        lat = lats[idx]
        lon = lons[idx]
        
        # Select snow depths within window boundries
        window_SD = SD_values[min_dist_row:max_dist_row]
        
        # Only Search for snow depths if there is enough Photons in window
        if len(window_SD) > 5: #Normally 5
                
            #Define new snow depth as median of snow depth within larger rescaled window boundries
            SD = np.median(window_SD)


            # Add center of window as new along track distance in element i.
            AT_window_center_arr[i] = window_center 
            
            # Add estimated surface
            snow_depth_arr[i] = SD
            
            # Add Latitude
            lat_arr[i] = lat
            
            # Add longitude
            lon_arr[i] = lon

            i+=1

        else:
            
            # If there is not enough photons in window, set surface to -999. 
            AT_window_center_arr[i] = window_center
            snow_depth_arr[i] = -999
            lat_arr[i] = lat
            lon_arr[i] = lon
            i+=1

    df = gpd.GeoDataFrame(AT_window_center_arr, columns = ['AT_dist']) 
    df["SD"] = snow_depth_arr
    df["lats"] = lat_arr
    df["lons"] = lon_arr

    gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.lons, df.lats))
    gdf = gdf.set_crs('EPSG:25833')
    
    #Filter bad observations
    gdf = gdf[gdf['SD'] > 0]
    gdf = gdf[gdf['SD'] < 10]
    gdf = gdf[gdf['SD'] != -999]
    if gdf.shape[0] > 0:
        outdir = path + date + '/SnowDepth/' + beam + '.shp'
        gdf.to_file(outdir)
    


In [ ]:
def Sample(path, date):
    
    i=0
    SD_path = path + date + '/SnowDepth/'
    for file in os.listdir(SD_path):
        if file.endswith('shp'):

            points = os.path.join(SD_path,file)
            pts = gpd.read_file(points)
            
            if i != 0:
                df_train = pd.concat([df_train,pts])
            else: 
                df_train = pts
                i=i+1  
                
    pts = df_train
    coords = [(x,y) for x, y in zip(pts.geometry.x, pts.geometry.y)]

    VH = path + date + '/S1/VH.tif'
    Diff = path + date + '/S1/Diff.tif'
    Ratio = path + date + '/S1/Ratio.tif'
    Subtract = path + date + '/S1/Subtract.tif'
    DEM = path[:-7] + '/DEM/mergedDEM10m_Hardanger.tif'
    
    outdir = path + date + '/Sampled/SD.shp'
    
    listofmasks=[VH, Diff, Ratio, Subtract, DEM]
    listofmasksname=['VH', 'Diff', 'Ratio', 'Subtract','DEM']
    i = 0
    for mask in listofmasks:
        src = rasterio.open(mask)
        pts[listofmasksname[i]] = [x[0] for x in src.sample(coords)]
        i=i+1
    pts = pts.drop(columns=['lats', 'lons', 'AT_dist'])    
    pts.to_file(outdir)
    return pts

In [ ]:
def run(params):

    #datelist = ['20200204','20200206','20200306','20200314','20200406','20200410','20210203','20210205','20210207','20210209','20210211','20210304','20210308']
    datelist = ['20210211','20210304']
    for dates in datelist:
        params['date'] = dates
        print('started processing of date: ', dates)
    
    
        #beamlist = ['gt1l','gt2l','gt3l','gt1r','gt2r','gt3r']
        beamlist = ['gt1l','gt2l']

    #    i=0
        for ISbeam in beamlist:
            params['beam'] = ISbeam
            df, geoid = load_ATL03(params['path'],params['date'], params['beam'])
            print('ATL03 Dataset loaded')

            df_surf = surface(df, 10, geoid, 10)
            df_surf = df_surf[df_surf['Surface'] != 0]
            print('Surface Heights Estimated')

            df_SD = snow_depth(df_surf, params['beam'], params['DEM'])
            print('Snow Depths Estimated')

            df_Mask = Mask(df_SD, params['path'], params['date'])
            print('Masked out training points')

            df_RE = Rescale(df_Mask, 3000, 1000, params['beam'], params['path'], params['date'])
            print('Rescaled training points')
            print('completed ICESat-2 Processing for beam: ',ISbeam)

            pts = Sample(params['path'], params['date'])
        
    return df, df_surf

In [ ]:
Load_params = {
    'path' : 'C:/Users/Rasmu/Documents/Thesis/Hardangervidda/Dates/',
    'date' : '20200306',
    #'DEM': 'C:/Users/Rasmu/Documents/Thesis/DEM/ArcticDEM_10m.tif',
    'DEM': 'C:/Users/Rasmu/Documents/Thesis/DEM/mergedDEM10m_Hardanger.tif'
    }

In [ ]:
df, df_surf = run(Load_params)